In [39]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img, img_to_array
import numpy as np

In [40]:
base_model = MobileNet(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(2, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='rmsprop', loss='binary_crossentropy') 

In [46]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    'train',
    target_size=(224, 224),
    batch_size=1,
    class_mode='categorical'
)

model.fit(train_generator, steps_per_epoch=1, epochs=40)

Found 18 images belonging to 2 classes.
Epoch 1/40
1/1 [==============================] - 0s 58ms/step - loss: 0.1433
Epoch 2/40
1/1 [==============================] - 0s 58ms/step - loss: 0.0568
Epoch 3/40
1/1 [==============================] - 0s 66ms/step - loss: 0.0814
Epoch 4/40
1/1 [==============================] - 0s 62ms/step - loss: 0.0634
Epoch 5/40
1/1 [==============================] - 0s 66ms/step - loss: 0.0801
Epoch 6/40
1/1 [==============================] - 0s 67ms/step - loss: 0.3168
Epoch 7/40
1/1 [==============================] - 0s 63ms/step - loss: 0.0729
Epoch 8/40
1/1 [==============================] - 0s 65ms/step - loss: 0.0397
Epoch 9/40
1/1 [==============================] - 0s 63ms/step - loss: 0.3694
Epoch 10/40
1/1 [==============================] - 0s 62ms/step - loss: 0.0330
Epoch 11/40
1/1 [==============================] - 0s 61ms/step - loss: 0.0117
Epoch 12/40
1/1 [==============================] - 0s 55ms/step - loss: 0.1165
Epoch 13/40
1/1 [====

In [47]:
bound = 0.85

def predict_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    predictions = model.predict(img_array)

    if predictions[0][0] < bound and predictions[0][1] < bound:
        return 'Nothing'
    elif predictions[0][0] >= bound:
        return 'box'
    elif predictions[0][1] >= bound:
        return 'tea'

In [56]:
predict_image('test/tea3.jpg')

'tea'

In [110]:
model.save("model.h5")

In [111]:
import tensorflow as tf

# 加载模型
model = tf.keras.models.load_model("model.h5")

# 转换模型
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# 保存 tflite 模型
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)


2024-05-13 14:26:39.357370: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmpst_d9oct/assets


2024-05-13 14:26:43.365329: I tensorflow/core/grappler/devices.cc:69] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2024-05-13 14:26:43.365480: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2024-05-13 14:26:43.366218: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:57:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 82 deviceMemorySize: 23.69GiB deviceMemoryBandwidth: 871.81GiB/s
2024-05-13 14:26:43.366822: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1871] Adding visible gpu devices: 0
2024-05-13 14:26:43.366855: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1258] Device interconnect StreamExecutor with strength 1 edge matrix:
2024-05-13 14:26:43.366858: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1264]      0 
2024-05-13 14:26:43.366861: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1277] 0:   N 
2024-05-13